In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from geopy import distance
import geopandas
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import pandas as pd
#import pygeohash
from sklearn.preprocessing import OneHotEncoder


from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from keras import metrics

import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, Reshape, BatchNormalization
from keras.utils import to_categorical
try:
    from keras.utils import np_utils
except:
    pass
try:
    from tensorflow.keras import utils as np_utils
except:
    pass

import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder

import platform

plat = platform.system()
print("Platform: ", plat)
if ("M2" in plat) or ("arm" in plat) or ("Darwin" in plat) or ("M1" in plat) or ("M3" in plat):
    is_Apple_silicon = True
    optimizer = tf.keras.optimizers.legacy.Adam()
    print("Apple Silicon")
else:
    is_Apple_silicon = False
    #optimizer = tf.optimizers.Adam()
    optimizer = tf.keras.optimizers.legacy.Adam()
    print("Non-Apple Silicon")

error_metric_used = 'mean_squared_error'


#from keras.utils import np_utils

Platform:  Windows
Non-Apple Silicon


In [2]:
#!pip install tensorflow[and-cuda]
#!pip install keras


In [3]:
# Helper to plot loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.legend()
  plt.grid(True)

# Assignment 4 - Simple Neural Networks

For this assigment you'll do a realistic task - predicting fraud from transaction data.
### Some Things to Note

<ul>
<li> The dataset is imbalanced. See: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data for some ideas
<li> The locations, time, dob all likely aren't super useful on their own, but can be made into something more useful without much code or trouble. Think about how it may be useful to represent them. The data doesn't have missing rows, so this is the main data prep portion.
<li> With respect to the above, and the other data here, we have a lot of rows of data. That means that we can generally handle data that is reasonably wide...
</ul>

### Deliverables

Your final goal is to produce a function that can be called to classify a transaction:
<ul>
<li> Please submit two .ipynb files - one where you did your work, and another that can use your model to make predictions.
<li> In that prediction file, please ensure:
    <ul>
    <li> You have a function where I can load a file, and the end result is a classificaiton matrix of your prediction accuracy.
    <li> You load a trained model. There's no training here.
    <li> Any data prep stuff that is needed for your data should be built in here. I'm going to run a test file that is the exact same setup as the training data.
    <li> I should be able to open the prediction file, load the test data, and click RUN ALL and things should work.
    <li> In addition to that, please include a short (~1-2 paragraph) description of what you did. Include anything that was innovative/different as well as a note on:
        <ul>
        <li> Any imbalanced data steps.
        <li> Treatment of the location and time variables. What did you do to them?
        <li> Model structure (layers/size)
        <li> Any optimization steps included - regularization, dropouts, feature selection, etc...
        </ul>
    </ul>
</ul>

### Grades

The grade breakdown is as follows:

<ul>
<li> Code preduces predictions - 40
<li> Accuracy - 30
<li> Explaination - 20
<li> Balance/variable transformations - 10
</ul>



In [4]:
# Load some data
df = pd.read_csv("https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/fraudTrain.csv.zip")
df.drop(columns={"Unnamed: 0"}, inplace=True)
df.head()


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [5]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
trans_date_trans_time,1296675,1274791,2019-04-22 16:02:01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cc_num,1296675.0,NaN,NaN,NaN,417192042079726656.0,1308806447000789248.0,60416207185.0,180042946491150.0,3521417320836166.0,4642255475285942.0,4992346398065154048.0
merchant,1296675,693,fraud_Kilback LLC,4403,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category,1296675,14,gas_transport,131659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amt,1296675.0,NaN,NaN,NaN,70.351035,160.316039,1.0,9.65,47.52,83.14,28948.9
first,1296675,352,Christopher,26669,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,1296675,481,Smith,28794,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gender,1296675,2,F,709863,NaN,NaN,NaN,NaN,NaN,NaN,NaN
street,1296675,983,0069 Robin Brooks Apt. 695,3123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,1296675,894,Birmingham,5617,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Deal with Lat/Lon

We can utilize lat/lon of the home and merchant in a useful way?

Note: I left the section headers in from when I did it. You can remove them if you want.

In [6]:
#get distance from transaction to merchant in km
df['distance'] = df.apply(lambda row: distance.distance((row['lat'], row['long']), (row['merch_lat'], row['merch_long'])).km, axis=1)

### Deal with Time

Can we make date/time and the date of birth into something useful?

In [7]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df["day_of_week"] = pd.to_datetime(df["trans_date_trans_time"]).dt.dayofweek
df['month'] =  df['trans_date_trans_time'].dt.month
df['year'] = df['trans_date_trans_time'].dt.year
#df["hour_of_day"] = pd.to_datetime(df["trans_date_trans_time"]).dt.hour
df['dob'] = pd.to_datetime(df['dob'])
df['age'] = (df['trans_date_trans_time'].dt.year - df['dob'].dt.year)

### Check Target Balance

In [8]:
pFraud = df['is_fraud'].mean()*100
print("percent fraud:", pFraud)

percent fraud: 0.5788651743883394


the percent of fraud is currently 0.58% which is extremely low

### Prepare Data


In [9]:
le = LabelEncoder()
df['merchant_encoded'] = le.fit_transform(df['merchant'])
df['category_encoded'] = le.fit_transform(df['category'])
df['state_encoded'] = le.fit_transform(df['state'])

In [10]:
columns_to_drop = [
    'trans_date_trans_time',
    'first',
    'last',
    'street',
    'city',
    'gender',
    'lat',
    'long',
    'merch_lat',
    'merch_long',
    'dob'  ,
    'job',
    'trans_num',
    'category',
    'merchant',
    'state']

df = df.drop(columns=columns_to_drop)

In [11]:
df.head(300)

,cc_num,amt,zip,city_pop,unix_time,is_fraud,distance,day_of_week,month,year,age,merchant_encoded,category_encoded,state_encoded
0,2703186189652095,4.97,28654,3495,1325376018,0,78.773821,1,1,2019,31,514,8,27
1,630423337322,107.23,99160,149,1325376044,0,30.216618,1,1,2019,41,241,4,47
2,38859492057661,220.11,83252,4154,1325376051,0,108.102912,1,1,2019,57,390,0,13
3,3534093764340240,45.00,59632,1939,1325376076,0,95.685115,1,1,2019,52,360,2,26
4,375534208663984,41.96,24433,99,1325376186,0,77.702395,1,1,2019,33,297,9,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3506042666828517,54.04,25049,630,1325389859,0,39.227891,1,1,2019,60,381,2,49
296,571844099986,909.66,40202,736284,1325389888,0,104.050231,1,1,2019,67,237,11,17
297,4045036286570100739,103.31,19310,3066,1325389940,0,50.621376,1,1,2019,91,533,4,38
298,4400011257587661852,105.38,68859,509,1325390047,0,74.977153,1,1,2019,39,650,3,29


### Prediction


In [13]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
def fraud_predict(data):
    dropped = data.drop('is_fraud', axis=1)  # Drop target column
    model = load_model('fraud_detect_ct')
    pred_prob = model.predict(dropped)  
    pred_labels = (pred_prob > 0.5).astype(int)  
    print(classification_report(data['is_fraud'], pred_labels))

fraud_predict(df)

40522/40522 [==============================] - 32s 788us/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1289169
           1       0.83      0.37      0.51      7506

    accuracy                           1.00   1296675
   macro avg       0.91      0.69      0.76   1296675
weighted avg       1.00      1.00      1.00   1296675

